#### Maven dependencies

In [1]:
%AddDeps org.apache.spark spark-mllib_2.10 1.6.2
%AddDeps com.github.haifengl smile-core 1.1.0 --transitive
%AddDeps io.reactivex rxscala_2.10 0.26.1 --transitive
%AddDeps com.chuusai shapeless_2.10 2.3.0 --repository https://oss.sonatype.org/content/repositories/releases/
%AddDeps org.tmoerman plongeur-spark_2.10 0.3.9 --repository file:/Users/tmo/.m2/repository
%AddDeps com.typesafe.play play-json_2.10 2.4.8

Marking org.apache.spark:spark-mllib_2.10:1.6.2 for download
Preparing to fetch from:
-> file:/var/folders/zz/zyxvpxvq6csfxvn_n0000000000000/T/toree_add_deps842518354258793019/
-> https://repo1.maven.org/maven2
-> New file at /var/folders/zz/zyxvpxvq6csfxvn_n0000000000000/T/toree_add_deps842518354258793019/https/repo1.maven.org/maven2/org/apache/spark/spark-mllib_2.10/1.6.2/spark-mllib_2.10-1.6.2.jar
Marking com.github.haifengl:smile-core:1.1.0 for download
Preparing to fetch from:
-> file:/var/folders/zz/zyxvpxvq6csfxvn_n0000000000000/T/toree_add_deps842518354258793019/
-> https://repo1.maven.org/maven2
-> New file at /var/folders/zz/zyxvpxvq6csfxvn_n0000000000000/T/toree_add_deps842518354258793019/https/repo1.maven.org/maven2/com/github/haifengl/smile-graph/1.1.0/smile-graph-1.1.0.jar
-> New file at /var/folders/zz/zyxvpxvq6csfxvn_n0000000000000/T/toree_add_deps842518354258793019/https/repo1.maven.org/maven2/com/github/haifengl/smile-data/1.1.0/smile-data-1.1.0.jar
-> New file at /va

In [5]:
%addjar http://localhost:8888/nbextensions/declarativewidgets/declarativewidgets.jar

Using cached version of declarativewidgets.jar


#### Import classes

In [6]:
import rx.lang.scala.{Observer, Subscription, Observable}
import rx.lang.scala.subjects.PublishSubject
import rx.lang.scala.subjects._

import shapeless.HNil

import org.tmoerman.plongeur.tda._
import org.tmoerman.plongeur.tda.Model._
import org.tmoerman.plongeur.tda.cluster.Clustering._
import org.tmoerman.plongeur.tda.cluster.Scale._

import declarativewidgets._
initWidgets

import declarativewidgets.WidgetChannels.channel

#### Import polymer elements

In [4]:
%%html
<link rel='import' href='urth_components/paper-slider/paper-slider.html' 
        is='urth-core-import' package='PolymerElements/paper-slider'>

In [5]:
%%html
<link rel='import' href='urth_components/paper-button/paper-button.html' 
        is='urth-core-import' package='PolymerElements/paper-button'>

In [4]:
%%html
<link rel='import' href='urth_components/plongeur-graph/plongeur-graph.html' 
        is='urth-core-import' package='tmoerman/plongeur-graph'>

#### Reactive TDA Machine

Instantiate a `PublishSubject`. This stream of `TDAParams` instances represents the input of a `TDAMachine`. The `PublishSubject` listens to changes and sets these to the channel `"ch_TDA_1"` under the `"params"` key.

*TODO: unsubscribe previous on re-evaluation*

In [7]:
val in = PublishSubject[TDAParams]

val in_sub = in.subscribe(p => channel("ch_TDA_1").set("params", p.toString))

Create an initial `TDAParams` instance. In the same cell, we submit the instance to the `PublishSubject`.

In [22]:
val tdaParams =
      TDAParams(
        lens = TDALens(
          Filter("feature" :: 0 :: HNil, 10, 0.5),
          Filter("feature" :: 1 :: HNil, 13, 0.5)),
        clusteringParams = ClusteringParams(),
        scaleSelection = histogram(10))

in.onNext(tdaParams)

Specify alternative parameters.

In [25]:
val tdaParams_ALT =
      TDAParams(
        lens = TDALens(
          Filter("feature" :: 0 :: HNil, 20, 0.6)),
        clusteringParams = ClusteringParams(),
        scaleSelection = histogram(10))

in.onNext(tdaParams_ALT)

For the sake of illustration, we create an html snippet that listens to changes on the `"ch_TDA_1"` channel and displays the value of the `"params"` key.

In [10]:
%%html
<template is='urth-core-bind' channel='ch_TDA_1'>  
    <div style='background: #FFB; padding: 10px;'>
        <span style='font-family: "Courier"'>[[params]]</span>
    </div>
</template>

[[params]]

Notice that when we evaluate the `TDAParams` instantiation cells, the output of the yellow box changes.

#### Inititalize rdd

In [11]:
import org.apache.spark.rdd.RDD
import org.apache.commons.lang.StringUtils.trim
import org.apache.spark.mllib.linalg.Vectors.dense

def readCircle(file: String) = 
    sc.
        textFile(file).
        map(_.split(",").map(trim)).
        zipWithIndex.
        map{ case (Array(x, y), idx) => dp(idx, dense(x.toDouble, y.toDouble))}         

In [12]:
val data_path = "/Users/tmo/Work/batiskav/projects/plongeur/scala/plongeur-spark/src/test/resources/data/"

val circle_1k_path = data_path + "circle.1k.csv"

val rdd = readCircle(circle_1k_path)

In [13]:
val ctx = TDAContext(sc, rdd)

In [14]:
val machine_out: Observable[(TDAParams, TDAResult)] = TDAMachine.run(ctx, in)

Turn a TDAResult into a json data structure.

In [15]:
def json(result: TDAResult) = {
    import play.api.libs.json._

    val r = scala.util.Random

    JsObject(
      Seq(
        "nodes" -> JsArray(
          result.clusters.map(c => {
            JsObject(Seq(
              "id"    -> JsString(c.id.toString),
              "label" -> JsString(c.id.toString),
              "size"  -> JsNumber(c.dataPoints.size),
              "x"     -> JsNumber(r.nextInt(100)),
              "y"     -> JsNumber(r.nextInt(100))))
          })),
        "edges" -> JsArray(
          result.edges.map(e => {
            val (from, to) = e.toArray match {case Array(f, t) => (f, t)}
            JsObject(Seq(
              "id"     -> JsString(s"$from--$to"),
              "source" -> JsString(from.toString),
              "target" -> JsString(to.toString)))
          }))))
  }

In [16]:
val out_sub = machine_out.subscribe(
    onNext = (t) => t match {case (p, r) => channel("ch_TDA_1").set("result", json(r).toString) },
    onError = (e) => println("Error in TDA machine: ", e))

In [17]:
//out_sub.unsubscribe()

Name: Syntax Error.
Message: 
StackTrace: 

In [19]:
%%html
<template is='urth-core-bind' channel='ch_TDA_1'>    
    <plongeur-graph data="{{result}}"></plongeur-graph>
</template>

In [35]:
%%html
<template is='urth-core-bind' channel='ch_TDA_1'>  
    <div style='background: #FBF; padding: 10px;'>
        <textarea rows="7" style='font-family: "Courier"; width: 100%;'>[[result]]</textarea>
    </div>
</template>

[[result]]